In [ ]:
!pip install langchain
!pip install langchain-community

In [8]:
from langchain_community.llms import Ollama
import sqlite3
from langchain_community.utilities import SQLDatabase

In [9]:
llm = Ollama(
    base_url='http://192.168.1.124:11434',
    model="codestral:latest", temperature=0
)

In [10]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


'[(8,)]'

In [11]:
from langchain.chains import create_sql_query_chain

In [12]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many track in Album named 'Fireball?"})
response

"SELECT COUNT(T1.TrackId) FROM Track AS T1 INNER JOIN Album AS T2 ON T1.AlbumId = T2.AlbumId WHERE T2.Title = 'Fireball'"

In [13]:
db.run(response)

'[(7,)]'

In [14]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many track in Album named 'Fireball'"}) #join 2 tables: track & album : works


'The answer to the user question is:\n\nThere are 7 tracks in the album named "Fireball".'